# 09 - ASML Real-Data End-to-End Capstone

This notebook integrates pricing, validation, implied volatility, hedging, model risk, and portfolio overlay on **ASML** using historical data since **2020-01-01**.

Goals:
- Use real historical data (ASML) with strict integrity checks.
- Triangulate option prices across BS / Binomial / MC / PDE.
- Compare historical hedging behavior and stochastic-vol model risk.
- Evaluate protective-put overlay on historical windows.
- Export tables/figures and generate a capstone markdown report.


> $S_t$ is adjusted close at day $t$; returns use $r_t = \log(S_t/S_{t-1})$.

In [2]:
%cd /content
!rm -rf interactive_portfolio_optimization
!git clone https://github.com/basarr/interactive_portfolio_optimization.git

/content
Cloning into 'interactive_portfolio_optimization'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 134 (delta 64), reused 55 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (134/134), 83.68 KiB | 1.78 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [3]:
from __future__ import annotations

from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    candidates = [
        Path.cwd(),
        Path.cwd().parent,
        Path('/content/interactive_portfolio_optimization'),
        Path('/content/Interactive_Portfolio_Optimization'),
    ]
    found = None
    for c in candidates:
        if (c / 'src').exists():
            found = c
            break
    if found is None:
        for pp in Path('/content').rglob('pyproject.toml'):
            cand = pp.parent
            if (cand / 'src').exists():
                found = cand
                break
    if found is not None:
        ROOT = found

if not (ROOT / 'src').exists():
    raise FileNotFoundError('Could not locate project root containing src/.')

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

for p in [
    ROOT / 'data' / 'raw',
    ROOT / 'data' / 'processed',
    ROOT / 'results' / 'tables',
    ROOT / 'results' / 'figures',
    ROOT / 'results' / 'logs',
    ROOT / 'results' / 'reports',
]:
    p.mkdir(parents=True, exist_ok=True)

print('ROOT:', ROOT)


ROOT: /content/interactive_portfolio_optimization


In [4]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.lib.stride_tricks import sliding_window_view

from src.config import config_dict
from src.utils import set_seed
from src.market_data import (
    fetch_prices_yfinance,
    compute_realized_vol,
    assert_price_data_ready,
    price_data_quality_report,
    price_gap_report,
)
from src.black_scholes import bs_call_price, bs_put_price
from src.binomial import price_european_binomial
from src.monte_carlo import mc_price_european_gbm_terminal
from src.pde_fd import fd_price_european_bs
from src.implied_vol import implied_vol
from src.hedging import simulate_delta_hedge_on_paths
from src.stoch_vol import simulate_heston_lite_paths
from src.plotting import plot_overlay_distribution
from src.metrics import summary_table

cfg = config_dict(fast_mode=False)
set_seed(cfg['SEED'])

TICKER = 'ASML'
START_DATE = '2020-01-01'
END_DATE = (pd.Timestamp.utcnow().normalize() + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
print('Ticker:', TICKER)
print('Date range:', START_DATE, 'to', END_DATE)


Ticker: ASML
Date range: 2020-01-01 to 2026-02-12


> **Math notation:** $\Delta t = 1$ trading day; data checks enforce unique ordered index and bounded calendar gaps.

In [6]:
def fetch_with_retries(ticker: str, start: str, end: str, retries: int = 3) -> pd.DataFrame:
    last_exc = None
    for attempt in range(1, retries + 1):
        try:
            px = fetch_prices_yfinance([ticker], start=start, end=end, interval='1d')
            if px.empty:
                raise ValueError('Empty data returned from yfinance.')
            if ticker not in px.columns:
                raise ValueError(f"Expected column '{ticker}', got {list(px.columns)}")
            return px[[ticker]].dropna()
        except Exception as exc:
            last_exc = exc
            print(f'Attempt {attempt} failed: {exc}')
    raise RuntimeError(f'Failed to fetch {ticker} after {retries} attempts.') from last_exc

prices = fetch_with_retries(TICKER, START_DATE, END_DATE)
assert_price_data_ready(prices, ticker=TICKER, start=START_DATE, min_obs=700, max_gap_days=5)

quality = price_data_quality_report(prices, ticker=TICKER)
large_gaps = price_gap_report(prices[[TICKER]], max_gap_days=5)

# Double-check coverage and recency.
first_date = prices.index.min()
last_date = prices.index.max()

def _to_naive_ts(x):
    ts = pd.Timestamp(x)
    return ts.tz_convert(None) if ts.tz is not None else ts

today_ref = _to_naive_ts(pd.Timestamp.utcnow()).normalize()
last_date_ref = _to_naive_ts(last_date)

assert first_date <= pd.Timestamp(START_DATE) + pd.Timedelta(days=7), 'Coverage does not start near 2020-01-01.'
assert (today_ref - last_date_ref) <= pd.Timedelta(days=10), f'Data is stale; latest date too old: {last_date_ref.date()}'
assert not prices[TICKER].isna().any(), 'Found NaN in ASML close series.'

prices.to_csv(ROOT / 'data' / 'raw' / 'asml_daily_since_2020.csv')
quality.to_csv(ROOT / 'results' / 'tables' / 'asml_data_quality_report.csv', index=False)
large_gaps.to_csv(ROOT / 'results' / 'tables' / 'asml_large_gap_report.csv', index=False)

print('Rows:', len(prices))
print('First date:', first_date.date())
print('Last date:', last_date.date())
print('Unexpected large gaps:', len(large_gaps))
quality


Rows: 1535
First date: 2020-01-02
Last date: 2026-02-10
Unexpected large gaps: 0


,ticker,n_obs,first_date,last_date,has_missing_values,is_index_monotonic,has_duplicate_index,max_calendar_gap_days
0,ASML,1535,2020-01-02,2026-02-10,False,True,False,4.0


In [7]:
fig, ax = plt.subplots(figsize=(10, 4.5))
ax.plot(prices.index, prices[TICKER], label='ASML Adj Close', linewidth=1.25)
ax.set_title('ASML Adjusted Close Since 2020')
ax.set_xlabel('Date')
ax.set_ylabel('Price (USD)')
ax.grid(alpha=0.3)
ax.legend()
fig.tight_layout()
fig.savefig(ROOT / 'results' / 'figures' / 'asml_price_history.png', dpi=150)
plt.close(fig)


> **Math notation:** realized volatility uses \(\hat\sigma_{t,w}=\sqrt{252}\,\mathrm{std}(r_{t-w+1:t})\) with \(r_t=\log(S_t/S_{t-1})\).

In [ ]:
returns = np.log(prices[TICKER]).diff().dropna()
rv21 = compute_realized_vol(returns, window=21)
rv63 = compute_realized_vol(returns, window=63)
rv252 = compute_realized_vol(returns, window=252)

assert not returns.empty, 'Returns series is empty.'
assert rv63.dropna().shape[0] > 200, 'Not enough realized-vol observations.'

feature_df = pd.DataFrame(
    {
        'price': prices[TICKER],
        'log_return': returns.reindex(prices.index),
        'rv21': rv21.reindex(prices.index),
        'rv63': rv63.reindex(prices.index),
        'rv252': rv252.reindex(prices.index),
    }
)
feature_df.to_csv(ROOT / 'results' / 'tables' / 'asml_realized_vol_features.csv')

fig, axes = plt.subplots(2, 1, figsize=(10, 7), sharex=True)
axes[0].plot(prices.index, prices[TICKER], color='tab:blue')
axes[0].set_title('ASML Price')
axes[0].set_ylabel('USD')
axes[0].grid(alpha=0.25)

axes[1].plot(rv21.index, rv21, label='RV 21d')
axes[1].plot(rv63.index, rv63, label='RV 63d')
axes[1].plot(rv252.index, rv252, label='RV 252d')
axes[1].set_title('Rolling Realized Volatility (Annualized)')
axes[1].set_ylabel('Vol')
axes[1].set_xlabel('Date')
axes[1].grid(alpha=0.25)
axes[1].legend()

fig.tight_layout()
fig.savefig(ROOT / 'results' / 'figures' / 'asml_realized_vol.png', dpi=150)
plt.close(fig)

feature_df.tail()


> **Math notation:** \(C_{BS}=S_0e^{-qT}N(d_1)-Ke^{-rT}N(d_2)\), with cross-checks against CRR, MC, and PDE.

In [ ]:
analysis_date = prices.index.max()
S0 = float(prices.loc[analysis_date, TICKER])
r = float(cfg['R'])
q = float(cfg['Q'])
T_option = 63.0 / 252.0
K = float(np.round(S0, 2))
sigma_est = float(rv63.dropna().iloc[-1])
sigma_est = float(np.clip(sigma_est, 0.05, 1.0))

assert np.isfinite(S0) and S0 > 0
assert np.isfinite(sigma_est) and sigma_est > 0

bs_call = bs_call_price(S0=S0, K=K, r=r, q=q, sigma=sigma_est, T=T_option)
binom_call = price_european_binomial(S0=S0, K=K, r=r, q=q, sigma=sigma_est, T=T_option, N=500, option_type='call')
mc_out = mc_price_european_gbm_terminal(
    S0=S0,
    K=K,
    r=r,
    q=q,
    sigma=sigma_est,
    T=T_option,
    n_paths=100_000,
    option_type='call',
    antithetic=True,
    seed=cfg['SEED'],
)
pde_out = fd_price_european_bs(
    S0=S0,
    K=K,
    r=r,
    q=q,
    sigma=sigma_est,
    T=T_option,
    option_type='call',
    S_max=3.0 * S0,
    M=280,
    N=280,
    scheme='CN',
)

pricing_rows = [
    {'method': 'Black-Scholes', 'price': bs_call},
    {'method': 'Binomial_CRR_N500', 'price': binom_call},
    {'method': 'MonteCarlo', 'price': mc_out['price'], 'ci_low': mc_out['ci_low'], 'ci_high': mc_out['ci_high']},
    {'method': 'PDE_CN', 'price': pde_out['price']},
]
pricing_df = pd.DataFrame(pricing_rows)
pricing_df['abs_error_vs_bs'] = (pricing_df['price'] - bs_call).abs()
pricing_df.to_csv(ROOT / 'results' / 'tables' / 'asml_pricing_crosscheck.csv', index=False)

fig, ax = plt.subplots(figsize=(9, 4.6))
ax.bar(pricing_df['method'], pricing_df['price'], alpha=0.8)
ax.axhline(bs_call, linestyle='--', color='black', label='BS reference')
mc_row = pricing_df[pricing_df['method'] == 'MonteCarlo'].iloc[0]
x_mc = pricing_df.index[pricing_df['method'] == 'MonteCarlo'][0]
ax.errorbar(
    x_mc,
    mc_row['price'],
    yerr=[[mc_row['price'] - mc_row['ci_low']], [mc_row['ci_high'] - mc_row['price']]],
    fmt='none',
    capsize=4,
    color='black',
)
ax.set_title('ASML Call Price Cross-Check Across Methods')
ax.set_ylabel('Option price')
ax.grid(alpha=0.25)
ax.legend()
fig.tight_layout()
fig.savefig(ROOT / 'results' / 'figures' / 'asml_pricing_crosscheck.png', dpi=150)
plt.close(fig)

pricing_df


> **Math notation:** implied volatility solves \(C_{BS}(\sigma_{imp}) = C_{mkt}\) for each strike.

In [ ]:
def fetch_option_snapshot(symbol: str, spot: float, min_days: int = 20):
    try:
        import yfinance as yf
    except ImportError:
        return pd.DataFrame(), None, 'no_yfinance'

    tk = yf.Ticker(symbol)
    expiries = list(tk.options)
    if len(expiries) == 0:
        return pd.DataFrame(), None, 'no_expiry'

    today = pd.Timestamp.utcnow().normalize()
    expiry_dates = pd.to_datetime(expiries)
    dte = (expiry_dates - today).days
    valid = np.where(dte >= min_days)[0]
    idx = int(valid[0]) if len(valid) > 0 else len(expiries) - 1
    expiry = expiries[idx]

    chain = tk.option_chain(expiry)
    calls = chain.calls.copy()
    cols = ['strike', 'bid', 'ask', 'lastPrice', 'volume', 'openInterest']
    available_cols = [c for c in cols if c in calls.columns]
    calls = calls[available_cols].copy()

    if {'bid', 'ask'}.issubset(calls.columns):
        calls['mid'] = np.where(
            (calls['bid'] > 0) & (calls['ask'] > 0),
            0.5 * (calls['bid'] + calls['ask']),
            calls['lastPrice'],
        )
    else:
        calls['mid'] = calls['lastPrice']

    calls = calls[calls['mid'] > 0].copy()
    calls = calls[(calls['strike'] >= 0.8 * spot) & (calls['strike'] <= 1.2 * spot)].copy()
    calls['option_type'] = 'call'
    calls['expiry'] = expiry
    calls['days_to_expiry'] = max(int((pd.Timestamp(expiry) - today).days), 1)
    return calls.sort_values('strike').reset_index(drop=True), expiry, 'yfinance_options'

opt_df, expiry_used, iv_source = fetch_option_snapshot(TICKER, S0)

if opt_df.empty:
    strikes = np.linspace(0.8 * S0, 1.2 * S0, 9)
    opt_df = pd.DataFrame({'strike': strikes, 'option_type': 'call'})
    opt_df['days_to_expiry'] = int(round(T_option * 365))
    opt_df['mid'] = [
        bs_call_price(S0=S0, K=float(k), r=r, q=q, sigma=sigma_est, T=T_option)
        for k in strikes
    ]
    iv_source = 'synthetic_fallback'

T_iv = float(opt_df['days_to_expiry'].iloc[0]) / 365.0
T_iv = max(T_iv, 5.0 / 365.0)

ivs = []
for _, row in opt_df.iterrows():
    try:
        iv = implied_vol(
            price=float(row['mid']),
            S=S0,
            K=float(row['strike']),
            r=r,
            q=q,
            T=T_iv,
            option_type='call',
        )
    except Exception:
        iv = np.nan
    ivs.append(iv)

opt_df = opt_df.copy()
opt_df['implied_vol'] = ivs
opt_df = opt_df.dropna(subset=['implied_vol']).reset_index(drop=True)
assert opt_df.shape[0] >= 5, 'Insufficient option points after IV inversion.'
opt_df['source'] = iv_source
opt_df.to_csv(ROOT / 'results' / 'tables' / 'asml_iv_snapshot.csv', index=False)

fig, ax = plt.subplots(figsize=(9, 4.6))
ax.plot(opt_df['strike'], opt_df['implied_vol'], marker='o', label=f'IV ({iv_source})')
ax.axhline(sigma_est, linestyle='--', color='black', label='63d realized vol')
ax.set_title(f'ASML Implied Volatility vs Strike | expiry={expiry_used or "synthetic"}')
ax.set_xlabel('Strike')
ax.set_ylabel('Implied volatility')
ax.grid(alpha=0.25)
ax.legend()
fig.tight_layout()
fig.savefig(ROOT / 'results' / 'figures' / 'asml_iv_smile_market.png', dpi=150)
plt.close(fig)

opt_df[['strike', 'mid', 'implied_vol', 'source']].head()


> **Math notation:** discrete hedge cost uses \(\mathrm{Cost}_t = c\,|\Delta_t-\Delta_{t^-}|\,S_t\), and hedging error is final hedged P\&L.

In [ ]:
horizon_steps = 21
close = prices[TICKER].dropna()
windows = sliding_window_view(close.to_numpy(), horizon_steps + 1)
assert windows.shape[0] > 400, 'Too few historical windows for hedging backtest.'

normalized_paths = 100.0 * windows / windows[:, [0]]
T_hedge = horizon_steps / 252.0
sigma_model = sigma_est

rebalance_list = [1, 5, 10]
tx_cost_list = [0.0, 0.001, 0.003]
hedge_rows = []
error_store = {}

for k in rebalance_list:
    for tx in tx_cost_list:
        out = simulate_delta_hedge_on_paths(
            paths=normalized_paths,
            K=100.0,
            r=r,
            q=q,
            sigma_model=sigma_model,
            T=T_hedge,
            rebalance_every_k_steps=k,
            option_type='call',
            tx_cost_per_dollar=tx,
        )
        hedge_rows.append(
            {
                'rebalance_every_k_steps': k,
                'tx_cost_per_dollar': tx,
                'mean_error': out['mean_error'],
                'std_error': out['std_error'],
                'q05': out['q05'],
                'q95': out['q95'],
            }
        )
        error_store[(k, tx)] = out['errors']

hedge_df = pd.DataFrame(hedge_rows)
hedge_df.to_csv(ROOT / 'results' / 'tables' / 'asml_hedging_backtest_summary.csv', index=False)

fig, ax = plt.subplots(figsize=(9, 4.6))
for tx, sub in hedge_df.groupby('tx_cost_per_dollar'):
    sub = sub.sort_values('rebalance_every_k_steps')
    ax.plot(sub['rebalance_every_k_steps'], sub['std_error'], marker='o', label=f'tx={tx:.3f}')
ax.set_title('ASML Historical-Window Hedging Tradeoff')
ax.set_xlabel('Rebalance every k days')
ax.set_ylabel('Hedging error std')
ax.grid(alpha=0.25)
ax.legend()
fig.tight_layout()
fig.savefig(ROOT / 'results' / 'figures' / 'asml_hedging_tradeoff.png', dpi=150)
plt.close(fig)

baseline_key = (5, 0.001)
baseline_errors = error_store[baseline_key]
fig, ax = plt.subplots(figsize=(9, 4.6))
ax.hist(baseline_errors, bins=60, density=True, alpha=0.8)
ax.set_title('ASML Hedging Error Distribution (k=5, tx=0.001)')
ax.set_xlabel('Hedging error')
ax.set_ylabel('Density')
ax.grid(alpha=0.2)
fig.tight_layout()
fig.savefig(ROOT / 'results' / 'figures' / 'asml_hedging_error_hist.png', dpi=150)
plt.close(fig)

hedge_df


> **Math notation:** stochastic-vol stress uses \(dV_t=\kappa(\theta-V_t)dt+\xi\sqrt{V_t}dW_t^{(2)}\), \(dS_t=(r-q)S_tdt+\sqrt{V_t}S_tdW_t^{(1)}\).

In [ ]:
n_sv_paths = min(10_000, normalized_paths.shape[0])
sv_paths = simulate_heston_lite_paths(
    S0=100.0,
    r=r,
    q=q,
    V0=cfg['V0'],
    kappa=cfg['KAPPA'],
    theta=cfg['THETA'],
    xi=cfg['XI'],
    rho=cfg['RHO'],
    T=T_hedge,
    n_paths=n_sv_paths,
    n_steps=horizon_steps,
    seed=cfg['SEED'],
)

sv_out = simulate_delta_hedge_on_paths(
    paths=sv_paths,
    K=100.0,
    r=r,
    q=q,
    sigma_model=sigma_model,
    T=T_hedge,
    rebalance_every_k_steps=5,
    option_type='call',
    tx_cost_per_dollar=0.001,
)

hist_out = simulate_delta_hedge_on_paths(
    paths=normalized_paths[:n_sv_paths],
    K=100.0,
    r=r,
    q=q,
    sigma_model=sigma_model,
    T=T_hedge,
    rebalance_every_k_steps=5,
    option_type='call',
    tx_cost_per_dollar=0.001,
)

model_risk_df = pd.DataFrame(
    [
        {'world': 'historical_windows', 'mean_error': hist_out['mean_error'], 'std_error': hist_out['std_error'], 'q05': hist_out['q05'], 'q95': hist_out['q95']},
        {'world': 'sv_misspecified_delta', 'mean_error': sv_out['mean_error'], 'std_error': sv_out['std_error'], 'q05': sv_out['q05'], 'q95': sv_out['q95']},
    ]
)
model_risk_df.to_csv(ROOT / 'results' / 'tables' / 'asml_model_risk_comparison.csv', index=False)

fig, axes = plt.subplots(1, 2, figsize=(10, 4.4))
axes[0].bar(model_risk_df['world'], model_risk_df['std_error'])
axes[0].set_title('Std of Hedging Error')
axes[0].tick_params(axis='x', rotation=20)
axes[0].grid(alpha=0.2)

axes[1].bar(model_risk_df['world'], model_risk_df['q05'])
axes[1].set_title('5% Quantile of Hedging Error')
axes[1].tick_params(axis='x', rotation=20)
axes[1].grid(alpha=0.2)

fig.tight_layout()
fig.savefig(ROOT / 'results' / 'figures' / 'asml_model_risk_comparison.png', dpi=150)
plt.close(fig)

model_risk_df


> **Math notation:** protective-put return is \(R_{hedged}=R_{unhedged}-\text{premium}+\max(K_{put}-S_T,0)\) in notional-normalized terms.

In [ ]:
overlay_horizon = 63
overlay_windows = sliding_window_view(close.to_numpy(), overlay_horizon + 1)
assert overlay_windows.shape[0] > 200, 'Too few windows for overlay analysis.'

start_prices = overlay_windows[:, 0]
end_prices = overlay_windows[:, -1]
start_index = close.index[: overlay_windows.shape[0]]

sigma_series = rv63.reindex(close.index).ffill().bfill()
sigma_start = np.clip(sigma_series.loc[start_index].to_numpy(), 0.05, 1.0)

K_put = 0.95 * start_prices
T_put = overlay_horizon / 252.0
premium = np.array(
    [
        bs_put_price(S=float(s), K=float(k), r=r, q=q, sigma=float(sig), T=T_put)
        for s, k, sig in zip(start_prices, K_put, sigma_start)
    ]
)

budget_frac = float(cfg['HEDGE_BUDGET_FRACTION'])
hedge_units = np.minimum(1.0, (budget_frac * start_prices) / premium)

unhedged_returns = end_prices / start_prices - 1.0
hedged_returns = (
    unhedged_returns
    - hedge_units * (premium / start_prices)
    + hedge_units * np.maximum(K_put - end_prices, 0.0) / start_prices
)

overlay_metrics = pd.concat(
    [
        summary_table(unhedged_returns).assign(strategy='unhedged').set_index('strategy'),
        summary_table(hedged_returns).assign(strategy='protective_put').set_index('strategy'),
    ],
    axis=0,
)
overlay_metrics['avg_premium_budget_fraction_used'] = [
    float(np.mean(hedge_units * premium / start_prices)),
    float(np.mean(hedge_units * premium / start_prices)),
]
overlay_metrics.to_csv(ROOT / 'results' / 'tables' / 'asml_overlay_metrics.csv')

plot_overlay_distribution(
    unhedged_returns,
    hedged_returns,
    ROOT / 'results' / 'figures' / 'asml_overlay_return_dist.png',
)

overlay_metrics


> **Math notation:** validation checks compare consistency and risk outcomes; e.g. MC check \(C_{BS}\in[CI_{low},CI_{high}]\).

In [ ]:
validation_rows = [
    {'check': 'data_non_empty', 'passed': bool(len(prices) > 700), 'detail': f'rows={len(prices)}'},
    {'check': 'data_start_near_2020', 'passed': bool(first_date <= pd.Timestamp(START_DATE) + pd.Timedelta(days=7)), 'detail': str(first_date.date())},
    {'check': 'data_no_nan', 'passed': bool(not prices[TICKER].isna().any()), 'detail': 'ASML series has no NaN'},
    {'check': 'data_no_large_gap_gt5d', 'passed': bool(large_gaps.empty), 'detail': f'large_gap_count={len(large_gaps)}'},
    {'check': 'mc_bs_inside_ci', 'passed': bool(mc_out['ci_low'] <= bs_call <= mc_out['ci_high']), 'detail': f"bs={bs_call:.4f}, ci=[{mc_out['ci_low']:.4f},{mc_out['ci_high']:.4f}]"},
    {'check': 'pde_close_to_bs', 'passed': bool(abs(pde_out['price'] - bs_call) < 0.75), 'detail': f"abs_error={abs(pde_out['price'] - bs_call):.4f}"},
    {'check': 'iv_points_available', 'passed': bool(opt_df.shape[0] >= 5), 'detail': f'iv_points={opt_df.shape[0]}'},
    {'check': 'sv_risk_exceeds_hist', 'passed': bool(model_risk_df.loc[1, 'std_error'] > model_risk_df.loc[0, 'std_error']), 'detail': f"hist_std={model_risk_df.loc[0,'std_error']:.4f}, sv_std={model_risk_df.loc[1,'std_error']:.4f}"},
    {'check': 'overlay_cvar_improves', 'passed': bool(overlay_metrics.loc['protective_put', 'cvar_95'] > overlay_metrics.loc['unhedged', 'cvar_95']), 'detail': f"unhedged={overlay_metrics.loc['unhedged','cvar_95']:.4f}, hedged={overlay_metrics.loc['protective_put','cvar_95']:.4f}"},
]
validation_df = pd.DataFrame(validation_rows)
validation_df.to_csv(ROOT / 'results' / 'tables' / 'asml_validation_checklist.csv', index=False)
validation_df


In [ ]:
summary_lines = [
    '# ASML Capstone Summary (Since 2020)',
    '',
    '## Scope',
    f'- Ticker: `{TICKER}`',
    f'- Start date: `{START_DATE}`',
    f'- End date used: `{END_DATE}`',
    '- Data source for historical prices: `yfinance`',
    f'- Option snapshot source: `{iv_source}`',
    '',
    '## Data Integrity',
    f'- Observations: `{len(prices)}`',
    f'- First date: `{first_date.date()}`',
    f'- Last date: `{last_date.date()}`',
    f'- Unexpected calendar gaps (>5 days): `{len(large_gaps)}`',
    '',
    '## Pricing Cross-Check (ATM-style call)',
    f'- BS: `{bs_call:.6f}`',
    f'- Binomial (N=500): `{binom_call:.6f}`',
    f"- Monte Carlo: `{mc_out['price']:.6f}` with 95% CI `[{mc_out['ci_low']:.6f}, {mc_out['ci_high']:.6f}]`",
    f"- PDE (CN): `{pde_out['price']:.6f}`",
    '',
    '## Hedging and Model Risk',
    f"- Historical-window hedge std (k=5, tx=0.001): `{hist_out['std_error']:.6f}`",
    f"- SV misspecified-delta hedge std: `{sv_out['std_error']:.6f}`",
    f"- Historical q05: `{hist_out['q05']:.6f}`",
    f"- SV q05: `{sv_out['q05']:.6f}`",
    '',
    '## Overlay Risk',
    f"- Unhedged CVaR95: `{overlay_metrics.loc['unhedged', 'cvar_95']:.6f}`",
    f"- Protective-put CVaR95: `{overlay_metrics.loc['protective_put', 'cvar_95']:.6f}`",
    f"- Unhedged std: `{overlay_metrics.loc['unhedged', 'std']:.6f}`",
    f"- Protective-put std: `{overlay_metrics.loc['protective_put', 'std']:.6f}`",
    '',
    '## Validation Checklist',
    'See `results/tables/asml_validation_checklist.csv`.',
    '',
    '## Figures',
    '![asml_price_history](../figures/asml_price_history.png)',
    '![asml_realized_vol](../figures/asml_realized_vol.png)',
    '![asml_pricing_crosscheck](../figures/asml_pricing_crosscheck.png)',
    '![asml_iv_smile_market](../figures/asml_iv_smile_market.png)',
    '![asml_hedging_tradeoff](../figures/asml_hedging_tradeoff.png)',
    '![asml_hedging_error_hist](../figures/asml_hedging_error_hist.png)',
    '![asml_model_risk_comparison](../figures/asml_model_risk_comparison.png)',
    '![asml_overlay_return_dist](../figures/asml_overlay_return_dist.png)',
    '',
    'If image rendering is unavailable, use these PNG names directly:',
    '`asml_price_history.png`, `asml_realized_vol.png`, `asml_pricing_crosscheck.png`,',
    '`asml_iv_smile_market.png`, `asml_hedging_tradeoff.png`, `asml_hedging_error_hist.png`,',
    '`asml_model_risk_comparison.png`, `asml_overlay_return_dist.png`.',
]
summary_md = "\n".join(summary_lines) + "\n"
summary_path = ROOT / 'results' / 'reports' / '09_asml_capstone_summary.md'
summary_path.write_text(summary_md, encoding='utf-8')
print('Wrote:', summary_path)
print('--- Preview ---')
print("\n".join(summary_md.splitlines()[:20]))
